# 1. Exploring `pandas`' Table Visualizations

_November 1st, 2021_

In [ ]:
!pip install --upgrade pandas

In [2]:
import pandas as pd 
pd.__version__

'1.3.4'

## **Issue #1: mouse hover**.

As you hover with your mouse over each row, [the line is not highlighted](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#Styler-Object-and-HTML).

In [3]:
import numpy as np

df = pd.DataFrame([[38.0, 2.0, 18.0, 22.0, 21, np.nan],[19, 439, 6, 452, 226,232]],
                  index=pd.Index(['Tumour (Positive)', 'Non-Tumour (Negative)'], name='Actual Label:'),
                  columns=pd.MultiIndex.from_product([['Decision Tree', 'Regression', 'Random'],['Tumour', 'Non-Tumour']], names=['Model:', 'Predicted:']))
df.style

[Similar to above, hovering with the mouse above each line does not highlight the line. Values are formatted as expected.](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#Formatting-Values)

In [4]:
df.style.format(precision=0, na_rep='MISSING', thousands=" ",
                formatter={('Decision Tree', 'Tumour'): "{:.2f}",
                           ('Regression', 'Non-Tumour'): lambda x: "$ {:,.1f}".format(x*-1e6)
                          })

## Issue #2: Color scheme is inconsistent.
[The color system is inconsistenty with the `Styler` object in the example](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#Hiding-Data), and -- similar to above -- rows are not highlighted upon mouse hover.

In [5]:
s = df.style.format('{:.0f}').hide_columns([('Random', 'Tumour'), ('Random', 'Non-Tumour')])
s

In [7]:
cell_hover = {  # for row hover use <tr> instead of <td>
    'selector': 'td:hover',
    'props': [('background-color', '#ffffb3')]
}
index_names = {
    'selector': '.index_name',
    'props': 'font-style: italic; color: darkgrey; font-weight:normal;'
}
headers = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #000066; color: white;'
}
s.set_table_styles([cell_hover, index_names, headers])

In [8]:
s.set_table_styles([
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.5em;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: bold;'},
], overwrite=False)

## Issue #3: No vertical demarcation between `Decision Tree` and `Regression`.

[See example just above this section.](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#Setting-Classes-and-Linking-to-External-CSS)

In [9]:
s.set_table_styles({
    ('Regression', 'Tumour'): [{'selector': 'th', 'props': 'border-left: 1px solid white'},
                               {'selector': 'td', 'props': 'border-left: 1px solid #000066'}]
}, overwrite=False, axis=0)

In [10]:
out = s.set_table_attributes('class="my-table-cls"').render()
print(out[out.find('<table'):][:109])

<table id="T_2e855_" class="my-table-cls">
  <thead>
    <tr>
      <th class="index_name level0" >Model:</th


In [11]:
s.set_table_styles([  # create internal CSS classes
    {'selector': '.true', 'props': 'background-color: #e6ffe6;'},
    {'selector': '.false', 'props': 'background-color: #ffe6e6;'},
], overwrite=False)
cell_color = pd.DataFrame([['true ', 'false ', 'true ', 'false '],
                           ['false ', 'true ', 'false ', 'true ']],
                          index=df.index,
                          columns=df.columns[:4])
s.set_td_classes(cell_color)

In [12]:
np.random.seed(0)
df2 = pd.DataFrame(np.random.randn(10,4), columns=['A','B','C','D'])
df2.style

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [13]:
def style_negative(v, props=''):
    return props if v < 0 else None
s2 = df2.style.applymap(style_negative, props='color:red;')\
              .applymap(lambda v: 'opacity: 20%;' if (v < 0.3) and (v > -0.3) else None)
s2

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [14]:
def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')
s2.apply(highlight_max, props='color:white;background-color:darkblue', axis=0)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [15]:
s2.apply(highlight_max, props='color:white;background-color:pink;', axis=1)\
  .apply(highlight_max, props='color:white;background-color:purple', axis=None)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [16]:
s.set_caption("Confusion matrix for multiple cancer prediction models.")\
 .set_table_styles([{
     'selector': 'caption',
     'props': 'caption-side: bottom; font-size:1.25em;'
 }], overwrite=False)

In [17]:
tt = pd.DataFrame([['This model has a very strong true positive rate',
                    "This model's total number of false negatives is too high"]],
                  index=['Tumour (Positive)'], columns=df.columns[[0,3]])
s.set_tooltips(tt, props='visibility: hidden; position: absolute; z-index: 1; border: 1px solid #000066;'
                         'background-color: white; color: #000066; font-size: 0.8em;'
                         'transform: translate(0px, -24px); padding: 0.6em; border-radius: 0.5em;')

In [18]:
s.set_table_styles([  # create internal CSS classes
    {'selector': '.border-red', 'props': 'border: 2px dashed red;'},
    {'selector': '.border-green', 'props': 'border: 2px dashed green;'},
], overwrite=False)
cell_border = pd.DataFrame([['border-green ', ' ', ' ', 'border-red '],
                           [' ', ' ', ' ', ' ']],
                          index=df.index,
                          columns=df.columns[:4])
s.set_td_classes(cell_color + cell_border)

In [19]:
df3 = pd.DataFrame(np.random.randn(4,4),
                   pd.MultiIndex.from_product([['A', 'B'], ['r1', 'r2']]),
                   columns=['c1','c2','c3','c4'])
df3

c1        c2        c3        c4
A r1 -1.048553 -1.420018 -1.706270  1.950775
  r2 -0.509652 -0.438074 -1.252795  0.777490
B r1 -1.613898 -0.212740 -0.895467  0.386902
  r2 -0.510805 -1.180632 -0.028182  0.428332

In [20]:
slice_ = ['c3', 'c4']
df3.style.apply(highlight_max, props='color:red;', axis=0, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [21]:
idx = pd.IndexSlice
slice_ = idx[idx[:,'r1'], idx['c2':'c4']]
df3.style.apply(highlight_max, props='color:red;', axis=0, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [22]:
slice_ = idx[idx[:,'r2'], :]
df3.style.apply(highlight_max, props='color:red;', axis=1, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [23]:
slice_ = idx[idx[(df3['c1'] + df3['c3']) < -2.0], ['c2', 'c4']]
df3.style.apply(highlight_max, props='color:red;', axis=1, subset=slice_)\
         .set_properties(**{'background-color': '#ffffb3'}, subset=slice_)

In [24]:
import seaborn as sns
cm = sns.light_palette("green", as_cmap=True)

df2.style.background_gradient(cmap=cm)

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [25]:
df2.style.bar(subset=['A', 'B'], color='#d65f5f')

,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [26]:
df2.style.bar(subset=['A', 'B'], align='mid', color=['#d65f5f', '#5fba7d'])


,A,B,C,D
0,1.764052,0.400157,0.978738,2.240893
1,1.867558,-0.977278,0.950088,-0.151357
2,-0.103219,0.410599,0.144044,1.454274
3,0.761038,0.121675,0.443863,0.333674
4,1.494079,-0.205158,0.313068,-0.854096
5,-2.552990,0.653619,0.864436,-0.742165
6,2.269755,-1.454366,0.045759,-0.187184
7,1.532779,1.469359,0.154947,0.378163
8,-0.887786,-1.980796,-0.347912,0.156349
9,1.230291,1.202380,-0.387327,-0.302303


In [27]:
def magnify():
    return [dict(selector="th",
                 props=[("font-size", "4pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]

In [28]:
np.random.seed(25)
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
bigdf = pd.DataFrame(np.random.randn(20, 25)).cumsum()

bigdf.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '1pt'})\
    .set_caption("Hover to magnify")\
    .format(precision=2)\
    .set_table_styles(magnify())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,0.23,1.03,-0.84,-0.59,-0.96,-0.22,-0.62,1.84,-2.05,0.87,-0.92,-0.23,2.15,-1.33,0.08,-1.25,1.20,-1.05,1.06,-0.42,2.29,-2.59,2.82,0.68,-1.58
1,-1.75,1.56,-1.13,-1.10,1.03,0.00,-2.46,3.45,-1.66,1.27,-0.52,-0.02,1.52,-1.09,-1.86,-1.13,-0.68,-0.81,0.35,-0.06,1.79,-2.82,2.26,0.78,0.44
2,-0.65,3.22,-1.76,0.52,2.20,-0.37,-3.00,3.73,-1.87,2.46,0.21,-0.24,-0.10,-0.78,-3.02,-0.82,-0.21,-0.23,0.86,-0.68,1.45,-4.89,3.03,1.91,0.61
3,-1.62,3.71,-2.31,0.43,4.17,-0.43,-3.86,4.16,-2.15,1.08,0.12,0.60,-0.89,0.27,-3.67,-2.71,-0.31,-1.59,1.35,-1.83,0.91,-5.80,2.81,2.11,0.28
4,-3.35,4.48,-1.86,-1.70,5.19,-1.02,-3.81,4.72,-0.72,1.08,-0.18,0.83,-0.22,-1.08,-4.27,-2.88,-0.97,-1.78,1.53,-1.80,2.21,-6.34,3.34,2.49,2.09
5,-0.84,4.23,-1.65,-2.00,5.34,-0.99,-4.13,3.94,-1.06,-0.94,1.24,0.09,-1.78,-0.11,-4.45,-0.85,-2.06,-1.35,0.80,-1.63,1.54,-6.51,2.80,2.14,3.77
6,-0.74,5.35,-2.11,-1.13,4.20,-1.85,-3.20,3.76,-3.22,-1.23,0.34,0.57,-1.82,0.54,-4.43,-1.83,-4.03,-2.62,-0.20,-4.68,1.93,-8.46,3.34,2.52,5.81
7,-0.44,4.69,-2.30,-0.21,5.93,-2.63,-1.83,5.46,-4.50,-3.16,-1.73,0.18,0.11,0.04,-5.99,-0.45,-6.20,-3.89,0.71,-3.95,0.67,-7.26,2.97,3.39,6.66
8,0.92,5.80,-3.33,-0.65,5.99,-3.19,-1.83,5.63,-3.53,-1.30,-1.61,0.82,-2.45,-0.40,-6.06,-0.52,-6.60,-3.48,-0.04,-4.60,0.51,-5.85,3.23,2.40,5.08
9,0.38,5.54,-4.49,-0.80,7.05,-2.64,-0.44,5.35,-1.96,-0.33,-0.80,0.26,-3.37,-0.82,-6.05,-2.61,-8.45,-4.45,0.41,-4.71,1.89,-6.93,2.14,3.00,5.16


In [29]:
bigdf = pd.DataFrame(np.random.randn(16, 100))
bigdf.style.set_sticky(axis="index")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-0.773866,-0.240521,-0.217165,1.173609,0.686390,0.008358,0.696232,0.173166,0.620498,0.504067,0.428066,-0.051824,0.719915,0.057165,0.562808,-0.369536,0.483399,0.620765,-0.354342,-1.469471,-1.937266,0.038031,-1.518162,-0.417599,0.386717,0.716193,0.489961,0.733957,0.914415,0.679894,0.255448,-0.508338,0.332030,-0.111107,-0.251983,-1.456620,0.409630,1.062320,-0.577115,0.718796,-0.399260,-1.311389,0.649122,0.091566,0.628872,0.297894,-0.142290,-0.542291,-0.914290,1.144514,0.313584,1.182635,1.214235,-0.416446,-1.653940,-2.550787,0.442473,0.052127,-0.464469,-0.523852,0.989726,-1.325539,-0.199687,-1.226727,0.290018,1.164574,0.817841,-0.309509,0.496599,0.943536,-0.091850,-2.802658,2.126219,-0.521161,0.288098,-0.454663,-1.676143,-0.357661,-0.788960,0.185911,-0.017106,2.454020,1.832706,-0.911743,-0.655873,-0.000514,-2.226997,0.677285,-0.140249,-0.408407,-0.838665,0.482228,1.243458,-0.477394,-0.220343,-2.463966,0.237325,-0.307380,1.172478,0.819492
1,0.405906,-0.978919,1.267526,0.145250,-1.066786,-2.114192,-1.128346,-1.082523,0.372216,0.004127,-0.211984,0.937326,-0.935890,-1.704118,0.611789,-1.030015,0.636123,-1.506193,1.736609,1.392958,1.009424,0.353266,0.697339,-0.297424,0.428702,-0.145346,-0.333553,-0.974699,0.665314,0.971944,0.121950,-1.439668,1.018808,1.442399,-0.199585,-1.165916,0.645656,1.436466,-0.921215,1.293906,-2.706443,1.460928,-0.823197,0.292952,-1.448992,0.026692,-0.975883,0.392823,0.442166,0.745741,1.187982,-0.218570,0.305288,0.054932,-1.476953,-0.114434,0.014103,0.825394,-0.060654,-0.413688,0.974836,1.339210,1.034838,0.040775,0.705001,0.017796,1.867681,-0.390173,2.285277,2.311464,-0.085070,-0.648115,0.576300,-0.790087,-1.183798,-1.334558,-0.454118,0.319302,1.706488,0.830429,0.502476,-0.079631,0.414635,0.332511,0.042935,-0.160910,0.918553,-0.292697,-1.303834,-0.199604,0.871023,-1.370681,-0.205701,-0.492973,1.123083,-0.081842,-0.118527,0.245838,-0.315742,-0.511806
2,0.011470,-0.036104,1.399603,-0.418176,-0.412229,-1.234783,-1.121500,1.196478,-0.569522,0.422022,-0.220484,0.804338,2.892667,-0.511055,-0.168722,-1.477996,-1.969917,0.471354,1.698548,0.137105,-0.762052,0.199379,-0.964346,-0.256692,1.265275,0.848762,-0.784161,1.863776,-0.355569,0.854552,0.768061,-2.075718,-2.501069,1.109868,0.957545,-0.683276,0.307764,0.733073,1.706250,-1.118091,0.374961,-1.414503,-0.524183,-1.662696,0.687921,0.521732,1.451396,-0.833491,-0.362796,-1.174444,-0.813893,-0.893220,0.770743,1.156647,-0.647444,0.125929,0.513600,-0.537874,1.992052,-1.946584,-0.104759,0.484779,-0.290936,-0.441075,0.542993,-1.050038,1.630482,0.239771,-1.177310,0.464804,-0.966995,0.646086,0.486899,1.022196,-2.267827,-1.229616,1.313805,1.073292,2.324940,-0.542720,-1.504292,0.777643,-0.618553,0.011342,1.385062,1.363552,-0.549834,0.688896,1.361288,-0.381137,0.797812,-1.128198,0.369208,0.540132,0.413853,-0.200308,-0.969126,0.981293,-0.009783,-0.320020
3,-0.574816,1.419977,0.434813,-1.101217,-1.586275,1.979573,0.378298,0.782326,2.178987,0.657564,0.683774,-0.091000,-0.059552,-0.738908,-0.907653,-0.701936,0.580039,-0.618757,0.453684,1.665382,-0.152321,0.880077,0.571073,-0.604736,0.532359,0.515031,-0.959844,-0.887184,0.435781,0.862093,-0.956321,-0.625909,0.194472,0.442490,0.526503,-0.215274,0.090711,0.932592,0.811999,-2.497026,0.631545,0.321418,-0.425549,-1.078832,0.753444,0.199790,-0.360526,-0.013448,-0.819476,0.814869,0.442118,-0.972048,-0.060603,-2.349825,1.265445,-0.573257,0.429124,1.049783,1.954773,0.071883,-0.094209,0.265616,0.948318,0.331645,1.343401,-0.167934,-1.105252,-0.167077,-0.096576,-0.838161,-0.208564,0.394534,0.762533,1.235357,-0.207282,-0.202946,-0.468025,0.256944,2.587584,1.186697,-1.031903,1.428316,0.658899,-0.046582,-0.075422,1.329359,-0.684267,-1.524182,2.014061,3.770933,0.64735

In [30]:
bigdf.index = pd.MultiIndex.from_product([["A","B"],[0,1],[0,1,2,3]])
bigdf.style.set_sticky(axis="index", pixel_size=18, levels=[1,2])